# **Group 4 Data Mining And Wrangling Test2**

### **Data source:** *https://www.imdb.com/chart/top/*
#### We used used different methods to get the get the data but we concluded with the *Top 250 Movies* from IMDB using Scrapy and spider

##### Load the data

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import missingno as mns 
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
import requests
import json
from datetime import date


### Since we saved the dataset as a JSON file, We had to convert it to csv format removing the the [] and [[]] formats (lists and nested lists)

In [ ]:

# Load the JSON file
json_file_path = "IMDB_250_movies.json"

with open(json_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Convert list values to plain values
for movie in data:
    for key, value in movie.items():
        if isinstance(value, list):
            if all(isinstance(i, list) for i in value):  # If nested lists exist, flatten them
                movie[key] = ", ".join([", ".join(map(str, sublist)) for sublist in value])
            else:
                movie[key] = ", ".join(map(str, value))

# Convert to DataFrame
df = pd.DataFrame(data)

# Save as CSV
csv_file_path = "IMDB_250_movies.csv"
df.to_csv(csv_file_path, index=False, encoding="utf-8")

csv_file_path


#### Reading the csv Dataset 

In [ ]:
## reading the csv dataset 
df = pd.read_csv("IMDB_250_movies.csv")
df.head(6)

## **Exploratory Data Analysis**

In [ ]:
## checking the shape of the dataset for information
df.info()

### Now we can use the OMDB API to get the other attributes of the dataset i.e. Main Actors, Budget, Box Office Gross, and Awards

In [ ]:
# OMDb API Key
OMDB_API_KEY = "60fded84"

# Function to fetch movie details from OMDb API
def fetch_movie_details(title):
    url = f"http://www.omdbapi.com/?t={title}&apikey={OMDB_API_KEY}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        # Check if the response is valid
        if data.get("Response") == "True":
            return {
                "title": data.get("Title", title),  # Ensure title matches
                "Cast (Main Actors)": data.get("Actors", "N/A"),
                "Budget": "N/A",  # OMDb does not provide budget info
                "Box Office Gross": data.get("BoxOffice", "N/A"),
                "Awards": data.get("Awards", "N/A")
            }
    
    # If no valid response, return empty data
    return {
        "title": title,
        "Cast (Main Actors)": "N/A",
        "Budget": "N/A",
        "Box Office Gross": "N/A",
        "Awards": "N/A"
    }

# Load your existing CSV file
existing_csv = "IMDB_250_movies.csv"
df = pd.read_csv(existing_csv)

# Fetch details for each movie
additional_data = [fetch_movie_details(title) for title in df["title"]]

# Convert the fetched data into a DataFrame
extra_df = pd.DataFrame(additional_data)

# Merge with the existing DataFrame
df = df.merge(extra_df, on="title", how="left")

# Save updated data
new_df_path = "Final_IMDB_250_movies_with_OMDB.csv"
df.to_csv(new_df_path, index=False, encoding="utf-8")

print(f"Updated CSV saved at: {new_df_path}")


## **Cleaning Data** 

In [ ]:
df1 = pd.read_csv("Final_IMDB_250_movies_with_OMDB.csv")

df1.columns


## **Visualizing the Missing Values**

In [ ]:
#### checking for missing values in the Box Office column
mns.bar(df1)

In [ ]:
#### checking for missing values in the Box Office column
missing_values = df1.isnull().sum()

missing_values

In [ ]:
### Dropping the Budget column
#The Budget column is not necessary for the analysis since it is only missing values, so we can drop it from the data frame.

df2 = df1.drop(columns=["Budget"])

df2.columns


In [ ]:
## checking the missing values in the dataset

df2.isnull().sum()

In [ ]:
### Dropping the missing values
#Since the missing values are few, we can drop them from the dataset.
df3 = df2.dropna()



In [ ]:
df3.head(6)

In [ ]:
# checking the missing values in the dataset again
df3.isna().sum()


## **Now we can save the data as a new dataset**

In [ ]:
# saving df3 to a new file
df3.to_csv("IMDB_250_movies_cleaned2.csv", index=False)


### Loading the cleaned data
cleaned_data = pd.read_csv('IMDB_250_movies_cleaned2.csv')
cleaned_data.head(4)

### Converting the release_day, month, and year to strings

In [ ]:
cleaned_data[['release_day','release_month','release_year']] = cleaned_data[['release_day','release_month','release_year']].astype('str')

In [ ]:
cleaned_data.dtypes

### Now convert the release day, month, and year to day-month-year

In [ ]:
cleaned_data['date'] = pd.to_datetime(cleaned_data['release_year'] + '-' +  cleaned_data['release_month'] + '-'+ cleaned_data['release_day'])
cleaned_data['date']

cleaned_data.head(4)

### convert the release_month to the month name

In [ ]:
cleaned_data['Release_Month_name'] = cleaned_data.date.dt.month_name()
cleaned_data.head(4)

### Getting the release_week_day

In [ ]:

cleaned_data['Release_week_day'] = cleaned_data.date.dt.day_name()

In [ ]:
## viewing the dataframe
cleaned_data.head(4)

In [ ]:
## column names
cleaned_data.columns

In [ ]:
cleaned_data['Release_Week_day'] = cleaned_data.Date.dt.day_name()
cleaned_data['Release_Month']    = cleaned_data.Date.dt.month_name()

In [ ]:
cleaned_data.head(3)

In [ ]:
cleaned_data.columns